In [1]:
import subprocess
import itertools
import random
from random import randrange
import statistics 

In [2]:
STUDENT_ID = 12021446

def check(student_id , passwords) :
    proc = subprocess.Popen(["./unlock64", str(student_id)] + passwords , stdout = subprocess.PIPE)
    results = []
    while True :
        line = proc.stdout.readline()
        if not line :
            break
        results.append(float(str(line).split("\\t")[1].split("\\r\\n")[0]))
    return results

print(check(STUDENT_ID , ["PASSWORD", "ALGOGEN"]))

[0.328869, 0.344896]


In [3]:
GENES = "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"

In [4]:
def generationIndividu(size):
    return [random.choice(GENES) for i in range(size)]

In [5]:
print(generationIndividu(18))

['I', 'B', '5', 'Z', '0', 'H', 'A', '3', 'V', '2', '4', '5', 'L', '1', '7', 'U', '7', 'Y']


In [6]:
#Le nombre d'individus à générer est passé en paramètre
def generationPopulation(nombreIndividus):
    return [generationIndividu(randrange(12,19)) for i in range(nombreIndividus)]

In [7]:
population = generationPopulation(10)

In [8]:
def mutation(enfants, prop_mutee = 1/ 100):
    for enfant in enfants :
        if random.random() < prop_mutee:
            indexGene = randrange(len(enfant))
            enfant[indexGene] = random.choice(GENES)
            

In [9]:
#Croisement des parents pour créer une nouvelle génération
def croisement( population, parents):
    nombreDeParentsSelectionnes = len(parents)
    nombreEnfantsSouhaites = len(population)-nombreDeParentsSelectionnes
    
    enfants = []
    while len(enfants) < nombreEnfantsSouhaites:
        pere = random.choice(parents)
        mere = random.choice(parents)
        #Le nombre de gènes du père sera égal à la longueur du code secret divisé par deux
        nombreGenesPere = len(pere)//2
        #Le nombre de gènes de la mère sera égal à la longueur du mot - le nombre de gène du père
        nombreGenesMere = len(mere)//2
        #print("nb gene :",nombreGenesPere +nombreGenesMere)
        enfant = pere[:nombreGenesPere] + mere[nombreGenesMere:]
        enfants.append(enfant)
    parents.extend(enfants)
    return enfants


In [10]:
def evaluation(population):
    list_evaluation = []
    for individu in population:
        list_evaluation.append((individu, check(STUDENT_ID , ["".join(individu)])[0]))
    sorted(list_evaluation, key=lambda x:x[1])
    return list_evaluation
    

In [11]:
def selection(list_evaluation, pourcentage_selec):
    individus_evaluations = []
    individus_score = []
    for individu, score in list_evaluation :
        if score >= 0.9 :
            return individu
        individus_evaluations.append(individu)
        individus_score.append(score)
        
    #Nombre des parents à garder
    nb_parent = int(len(individus_evaluations)*pourcentage_selec)
    parents = individus_evaluations[:nb_parent]
          
    proba = 0
    index = nb_parent
    for individu in individus_evaluations[nb_parent:]:
        proba = proba + (individus_score[index]/sum(individus_score))
        if random.random() < proba:
            parents.append(individu)
        index = index +1
    return parents

In [ ]:
NB_INDIVIDUS = 100
PROP_MUTATION = 1/100
MAX_GENERATIONS = 10000
POURCENTAGE_SELECTION = 0.3

population = generationPopulation(NB_INDIVIDUS)
i = 0
score = 0
fitness = []
while i < MAX_GENERATIONS and score <= 0.9 :
    list_evaluation = evaluation(population)
    individus_score = []
    for individu, score in list_evaluation :
        individus_score.append(score)
    fitness.append((max(individus_score), statistics.mean(individus_score)))
    parents = selection(list_evaluation, POURCENTAGE_SELECTION)
    enfants = croisement( population, parents)
    mutation(enfants, prop_mutee = 1/ 100)
    population.extend(enfants)
    print("score max: ",fitness[i][0])
    print("score moyen: ",fitness[i][1])
    i = i + 1

nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 16
nb gene : 17
score max:  0.539811
score moyen:  0.4717803
nb gene : 18
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 15
score max:  0.539811
score moyen:  0.4649918666666667
nb gene : 15
nb gene : 17
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 14
nb gene : 16
score max:  0.54732
score moyen:  0.4688992380952381
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 15
score max:  0.592942
score moyen:  0.47859193333333333
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 18
nb gene : 16
nb gene : 17
nb gene : 14
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 14
nb gene : 16
nb gene : 16
score max:  0.603357
score moyen:  0.4797217209302326
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 16
nb gene : 1

nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 17
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 14
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15

nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16

nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 14
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16

nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 17

nb gene : 17
nb gene : 15
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 14
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 17
nb gene : 16
nb gene : 17
nb gene : 15
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 17
nb gene : 17
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 15
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16
nb gene : 16